# 🧱 Base Notebook: Transformers Feature Engineering

Este notebook prepara datasets para Transformer por horizonte (`W12`, `W18`, `W24`) uniendo:
- features base de `3_features` (day0 opcional),
- salidas de clustering (`p_cluster_*`, `confidence`, `entropy`),
- target final.

In [1]:
# =========================
# 1) Imports + Config
# =========================
from pathlib import Path
import json
import numpy as np
import pandas as pd

try:
    from educational_ai_analytics.config import (
        FEATURES_DATA_DIR, EMBEDDINGS_DATA_DIR, DATA_DIR, W_WINDOWS
    )
except Exception:
    ROOT = Path('/workspace/TFM_education_ai_analytics')
    DATA_DIR = ROOT / 'data'
    FEATURES_DATA_DIR = DATA_DIR / '3_features'
    EMBEDDINGS_DATA_DIR = DATA_DIR / '4_embeddings'
    W_WINDOWS = [12, 18, 24]

OUT_ROOT = DATA_DIR / '6_transformer_features'
HORIZONS = sorted([int(w) for w in W_WINDOWS])
SPLITS = ['training', 'validation', 'test']
INCLUDE_DAY0 = True
SEG_FILE = 'segmentation_gmm_ae.csv'
TARGET_FILE = 'target.csv'

OUT_ROOT.mkdir(parents=True, exist_ok=True)
print('✅ Config cargada')
print('Horizontes:', HORIZONS)
print('Output:', OUT_ROOT)

✅ Config cargada
Horizontes: [12, 18, 24]
Output: /workspace/TFM_education_ai_analytics/data/6_transformer_features


In [ ]:
# =========================
# 2) Helpers
# =========================
def safe_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame()
    return pd.read_csv(path, index_col=0).replace([np.inf, -np.inf], np.nan).fillna(0.0)

def load_target(split: str) -> pd.DataFrame:
    tgt = safe_read_csv(FEATURES_DATA_DIR / split / TARGET_FILE)
    if tgt.empty:
        raise FileNotFoundError(f'No target found for split={split}')
    return tgt

def load_day0(split: str) -> pd.DataFrame:
    return safe_read_csv(FEATURES_DATA_DIR / split / 'day0_static_features.csv')

def load_seg(split: str, w: int) -> pd.DataFrame:
    p = EMBEDDINGS_DATA_DIR / split / f'upto_w{w:02d}' / SEG_FILE
    seg = safe_read_csv(p)
    if seg.empty:
        return seg
    cols = [c for c in seg.columns if c.startswith('p_cluster_')]
    extra = [c for c in ['cluster_id', 'confidence', 'entropy', 'entropy_norm'] if c in seg.columns]
    keep = cols + extra
    seg = seg[keep].copy()
    seg.columns = [f'{c}_w{w:02d}' for c in seg.columns]
    return seg

def build_horizon_table(split: str, horizon: int, include_day0: bool = True) -> pd.DataFrame:
    windows = [w for w in HORIZONS if w <= horizon]
    tgt = load_target(split)

    base = tgt[['final_result']].copy()
    if include_day0:
        day0 = load_day0(split)
        if not day0.empty:
            day0 = day0.add_prefix('d0_')
            base = base.join(day0, how='inner')

    for w in windows:
        seg = load_seg(split, w)
        if seg.empty:
            print(f'⚠️ split={split} W{w}: segmentation vacía/no encontrada')
            continue
        base = base.join(seg, how='inner')

    base = base.dropna()
    return base

def export_horizon_dataset(df: pd.DataFrame, split: str, horizon: int):
    out_dir = OUT_ROOT / split / f'w{horizon:02d}'
    out_dir.mkdir(parents=True, exist_ok=True)

    y = df['final_result'].astype(int).values
    feature_df = df.drop(columns=['final_result'])

    prob_cols = [c for c in feature_df.columns if c.startswith('p_cluster_')]
    static_cols = [c for c in feature_df.columns if c not in prob_cols]

    windows = [w for w in HORIZONS if w <= horizon]
    seq_steps = []
    for w in windows:
        step_cols = [c for c in prob_cols if c.endswith(f'_w{w:02d}')]
        if step_cols:
            seq_steps.append(feature_df[step_cols].values.astype(np.float32))

    if len(seq_steps) == 0:
        X_seq = np.zeros((len(feature_df), 1, 1), dtype=np.float32)
    else:
        X_seq = np.stack(seq_steps, axis=1)

    X_static = feature_df[static_cols].values.astype(np.float32) if len(static_cols) else np.zeros((len(feature_df), 0), dtype=np.float32)

    np.save(out_dir / 'X_seq.npy', X_seq)
    np.save(out_dir / 'X_static.npy', X_static)
    np.save(out_dir / 'y.npy', y)

    feature_df.to_csv(out_dir / 'features_flat.csv')
    pd.DataFrame(index=df.index).to_csv(out_dir / 'index.csv')

    meta = {
        'split': split,
        'horizon': int(horizon),
        'n_samples': int(len(df)),
        'seq_shape': list(X_seq.shape),
        'static_shape': list(X_static.shape),
        'windows_used': windows,
        'include_day0': bool(any(c.startswith('d0_') for c in static_cols)),
        'n_prob_features_flat': int(len(prob_cols)),
        'n_static_features': int(len(static_cols))
    }
    (out_dir / 'meta.json').write_text(json.dumps(meta, indent=2, ensure_ascii=False))

    print(f'💾 Exportado {split} W{horizon}: N={len(df)} | seq={X_seq.shape} | static={X_static.shape}')

In [1]:
import tensorflow as tf
from tensorflow.keras import layers


class PositionalEncoding(layers.Layer):
    def __init__(self, d_model: int, max_len: int = 512):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        pos = tf.range(max_len, dtype=tf.float32)[:, tf.newaxis]  # (L, 1)
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]    # (1, D)

        angle_rates = 1.0 / tf.pow(10000.0, (2.0 * tf.floor(i / 2.0)) / tf.cast(d_model, tf.float32))
        angle_rads = pos * angle_rates

        # even -> sin, odd -> cos
        sin_part = tf.sin(angle_rads[:, 0::2])
        cos_part = tf.cos(angle_rads[:, 1::2])

        pe = tf.concat([sin_part, cos_part], axis=-1)
        pe = pe[tf.newaxis, ...]  # (1, L, D)
        self.pe = tf.cast(pe, tf.float32)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pe[:, :seq_len, :]


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, d_model: int, num_heads: int, ff_dim: int, dropout: float):
        super().__init__()
        self.mha = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,
            dropout=dropout,
        )
        self.dropout1 = layers.Dropout(dropout)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.add1 = layers.Add()

        self.ffn = tf.keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dropout(dropout),
                layers.Dense(d_model),
            ]
        )
        self.dropout2 = layers.Dropout(dropout)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.add2 = layers.Add()

    def call(self, x, training=False, attention_mask=None):
        attn_out = self.mha(
            query=x,
            value=x,
            key=x,
            attention_mask=attention_mask,
            training=training,
        )
        attn_out = self.dropout1(attn_out, training=training)
        x = self.add1([x, attn_out])
        x = self.norm1(x)

        ffn_out = self.ffn(x, training=training)
        ffn_out = self.dropout2(ffn_out, training=training)
        x = self.add2([x, ffn_out])
        x = self.norm2(x)
        return x


class GLULayer(layers.Layer):
    # h(X) = (XW) ⊗ sigmoid(XV)
    def __init__(self, d_model: int):
        super().__init__()
        self.proj = layers.Dense(2 * d_model)

    def call(self, x):
        z = self.proj(x)
        a, b = tf.split(z, num_or_size_splits=2, axis=-1)
        return a * tf.sigmoid(b)


class GLUTransformerClassifier(tf.keras.Model):
    def __init__(
        self,
        latent_d: int,
        num_heads: int,
        ff_dim: int,
        dropout: float,
        num_classes: int,
        num_layers: int = 2,
        max_len: int = 512,
    ):
        super().__init__()

        if latent_d % num_heads != 0:
            raise ValueError("latent_d debe ser divisible por num_heads")

        self.input_proj = layers.Dense(latent_d)
        self.pos_encoding = PositionalEncoding(d_model=latent_d, max_len=max_len)
        self.input_dropout = layers.Dropout(dropout)

        self.encoders = [
            TransformerEncoderBlock(
                d_model=latent_d,
                num_heads=num_heads,
                ff_dim=ff_dim,
                dropout=dropout,
            )
            for _ in range(num_layers)
        ]

        self.glu = GLULayer(d_model=latent_d)
        self.norm_out = layers.LayerNormalization(epsilon=1e-6)

        self.head = tf.keras.Sequential(
            [
                layers.Dense(latent_d, activation="relu"),
                layers.Dropout(dropout),
                layers.Dense(num_classes, activation="softmax"),
            ]
        )

    def call(self, inputs, training=False, mask=None):
        # inputs: (B, W, F)
        # mask:   (B, W) con 1/0 o bool (opcional)

        x = self.input_proj(inputs)          # (B, W, D)
        x = self.pos_encoding(x)             # (B, W, D)
        x = self.input_dropout(x, training=training)

        # Máscara para MHA: (B, 1, W), broadcast a (B, W, W)
        attn_mask = None
        if mask is not None:
            attn_mask = tf.cast(mask[:, tf.newaxis, :], tf.bool)

        for encoder in self.encoders:
            x = encoder(x, training=training, attention_mask=attn_mask)

        x = self.glu(x)
        x = self.norm_out(x)

        # Masked pooling temporal
        if mask is not None:
            m = tf.cast(mask, x.dtype)[:, :, tf.newaxis]     # (B, W, 1)
            x_sum = tf.reduce_sum(x * m, axis=1)             # (B, D)
            denom = tf.reduce_sum(m, axis=1) + 1e-8          # (B, 1)
            pooled = x_sum / denom
        else:
            pooled = tf.reduce_mean(x, axis=1)

        return self.head(pooled)


2026-02-19 17:45:51.241461: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr